In [27]:
import pandas as pd
from prophet import Prophet
from pymongo import MongoClient
from prophet.plot import plot_plotly, plot_components_plotly
import os
import time
import pickle
from datetime import datetime
import itertools
import dask
from dask.distributed import Client
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

In [32]:
df_clusters = pd.read_csv('~/ucc-21/clusters-covid.csv')
parents = list(df_clusters[df_clusters['is_parent'] == 1]['GISJOIN'])
print(parents)
# pickle.dump(parents, open('ucc-21/covid-parents.pkl', 'wb'))

['G0400130', 'G0400190', 'G0400270', 'G0500590', 'G0500690', 'G0501030', 'G0600370', 'G0600590', 'G0600650', 'G0600710', 'G0600730', 'G0800050', 'G0800150', 'G0900090', 'G1200090', 'G1200110', 'G1200170', 'G1200830', 'G1201050', 'G1301350', 'G1700310', 'G1700430', 'G1701110', 'G1800970', 'G1900610', 'G1901010', 'G2405100', 'G2601210', 'G2700810', 'G2900950', 'G3000130', 'G3200030', 'G3300150', 'G3400030', 'G3500010', 'G3600510', 'G3600590', 'G3600910', 'G3900170', 'G3900350', 'G3900410', 'G4001090', 'G4200030', 'G4201010', 'G4600990', 'G4800270', 'G4800290', 'G4800850', 'G4801090', 'G4801130', 'G4801570', 'G4802010', 'G4802150', 'G4900350', 'G5100090', 'G5400550']


In [30]:
for p in parents:
    print(p, end=', ')

G0400130, G0400190, G0400270, G0500590, G0500690, G0501030, G0600370, G0600590, G0600650, G0600710, G0600730, G0800050, G0800150, G0900090, G1200090, G1200110, G1200170, G1200830, G1201050, G1301350, G1700310, G1700430, G1701110, G1800970, G1900610, G1901010, G2405100, G2601210, G2700810, G2900950, G3000130, G3200030, G3300150, G3400030, G3500010, G3600510, G3600590, G3600910, G3900170, G3900350, G3900410, G4001090, G4200030, G4201010, G4600990, G4800270, G4800290, G4800850, G4801090, G4801130, G4801570, G4802010, G4802150, G4900350, G5100090, G5400550, 

In [3]:
df_map = pickle.load(open('pickles/covid/df_map.pkl', 'rb'))
df_cases = df_map['cases']
df_cases

,ds,y
0,2021-01-07,1757
1,2020-06-08,110
2,2020-10-25,265
3,2020-07-20,659
4,2020-12-27,6043
...,...,...
367,2020-03-14,2
368,2020-02-26,0
369,2020-03-06,0
370,2020-02-08,0


In [4]:
def predict(df_train):
    m = Prophet(
        daily_seasonality=True,
        weekly_seasonality=True,
        yearly_seasonality=True,
    )
    # model.fit(df, algorithm='LBFGS')
    fit_m = m.fit(df_train, algorithm='LBFGS')
    df_train_future = m.make_future_dataframe(periods=300)
    df_train_forecast = m.predict(df_train_future)

    return fit_m, df_train_future, df_train_forecast

# fit_m, df_train, df_train_forecast = predict(df_cases)

## Exhaustively train parents

In [12]:
db = MongoClient("lattice-100", 27018)
collection = 'covid_county_formatted'

def get_df_by_gis_join(gis_join):
    print(gis_join, end=' ')
    cursor = db.sustaindb[collection].aggregate([{"$match": {"GISJOIN": gis_join}}])
    df = pd.DataFrame(list(cursor))[['date', 'cases']]
    df.columns = ['ds','y']
    return df

def predict_parallel(df_train, gis_join):
    time1 = time.monotonic()
    m = Prophet(
        daily_seasonality=True,
        weekly_seasonality=True,
        yearly_seasonality=True,
    )
    fit_m = m.fit(df_train, algorithm='LBFGS')
    df_train_future = m.make_future_dataframe(periods=300, freq='H')
    df_train_forecast = m.predict(df_train_future)
    time2 = time.monotonic()

    return gis_join, (time2 - time1)


gis_joins_list = []
child_dfs_list = []
# cutoff = 3000

for parent in parents:
    gis_joins_list.append(parent)
    child_dfs_list.append(get_df_by_gis_join(parent))


def train_one_parent(df_, gis_join):
    gis_join, time_taken = predict_parallel(df_, gis_join)
    return gis_join, time_taken


# assert len(gis_joins_list) == len(parents)
print(f'#gis_joins: {len(gis_joins_list)}')
print(f'#child_dfs: {len(child_dfs_list)}')
assert len(gis_joins_list) == len(child_dfs_list)

G0400130 G0400190 G0400270 G0500590 G0500690 G0501030 G0600370 G0600590 G0600650 G0600710 G0600730 G0800050 G0800150 G0900090 G1200090 G1200110 G1200170 G1200830 G1201050 G1301350 G1700310 G1700430 G1701110 G1800970 G1900610 G1901010 G2405100 G2601210 G2700810 G2900950 G3000130 G3200030 G3300150 G3400030 G3500010 G3600510 G3600590 G3600910 G3900170 G3900350 G3900410 G4001090 G4200030 G4201010 G4600990 G4800270 G4800290 G4800850 G4801090 G4801130 G4801570 G4802010 G4802150 G4900350 G5100090 G5400550 #gis_joins: 56
#child_dfs: 56


In [17]:
client = Client('lattice-150:8786')

counter = 1
lazy_results = []
for gis_join, df_ in zip(gis_joins_list, child_dfs_list):
    lazy_result = dask.delayed(train_one_parent)(df_, gis_join)
    lazy_results.append(lazy_result)
    print(counter, end=', ')
    counter += 1

futures = dask.persist(*lazy_results)  # trigger computation in the background
results = dask.compute(*futures)
results[:5]

1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 

(('G0400130', 6.884672166081145),
 ('G0400190', 10.578587840078399),
 ('G0400270', 10.03168568504043),
 ('G0500590', 6.455884965136647),
 ('G0500690', 6.517799691064283))

## GridSearch on Parent Models

In [18]:
def grid_search(df):
    param_grid = {  
        'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
        'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
    }

    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    rmses = []  # Store the RMSEs for each params here

    time1 = time.monotonic()
    # Use cross validation to evaluate all parameters
    for params in all_params:
        m = Prophet(**params).fit(df)  # Fit model with given params
        df_cv = cross_validation(m, initial= '100 days', period='100 days', horizon='100', parallel="dask")
        df_p = performance_metrics(df_cv, rolling_window=1)
        rmses.append(df_p['rmse'].values[0])

    # Find the best parameters
    tuning_results = pd.DataFrame(all_params)
    tuning_results['rmse'] = rmses

    print(f'Time: {time.monotonic() - time1}')
    
    optimal_params = tuning_results[tuning_results['rmse'] == min(tuning_results['rmse'])]
    rmse = optimal_params['rmse'].iloc[0]
    changepoint_prior_scale = optimal_params['changepoint_prior_scale'].iloc[0]
    seasonality_prior_scale = optimal_params['seasonality_prior_scale'].iloc[0]
    
    return rmse, changepoint_prior_scale, seasonality_prior_scale


rmses = []
changepoint_prior_scales = []
seasonality_prior_scales = []

counter = 1
for parent in parents:
    df_parent = get_df_by_gis_join(parent)
    rmse, changepoint_prior_scale, seasonality_prior_scale = grid_search(df_parent)
    rmses.append(rmse)
    changepoint_prior_scales.append(changepoint_prior_scale)
    seasonality_prior_scales.append(seasonality_prior_scale)
    print(counter, end=', ')
    counter += 1

G0400130 

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2020-05-10 23:59:59.999999900 and 2021-03-06 23:59:59.999999900


Initial log joint probability = -6.7817
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
       1       -6.7817             0       406.423       1e-12       0.001       25   
Optimization terminated with error: 
  Line search failed to achieve a sufficient decrease, no more progress can be made

Initial log joint probability = -208.59
Iteration  1. Log joint probability =    388.567. Improved by 597.156.
Iteration  2. Log joint probability =    595.769. Improved by 207.203.
Iteration  3. Log joint probability =    633.315. Improved by 37.5457.
Iteration  4. Log joint probability =    636.665. Improved by 3.3502.
Iteration  5. Log joint probability =    636.854. Improved by 0.188657.
Iteration  6. Log joint probability =    637.116. Improved by 0.261877.
Iteration  7. Log joint probability =    637.158. Improved by 0.0425729.
Iteration  8. Log joint probability =    637.378. Improved by 0.219796.
Iteration  9. Log joint probability =    637.88

INFO:prophet:Applying in parallel with <Client: 'tcp://129.82.208.160:8786' processes=24 threads=384, memory=1.47 TiB>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2020-05-10 23:59:59.999999900 and 2021-03-06 23:59:59.999999900


Initial log joint probability = -6.7817
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
       1       -6.7817             0       406.423       1e-12       0.001       25   
Optimization terminated with error: 
  Line search failed to achieve a sufficient decrease, no more progress can be made

Initial log joint probability = -222.405
Iteration  1. Log joint probability =    355.207. Improved by 577.612.
Iteration  2. Log joint probability =    516.996. Improved by 161.789.
Iteration  3. Log joint probability =    559.583. Improved by 42.5869.
Iteration  4. Log joint probability =     605.28. Improved by 45.697.
Iteration  5. Log joint probability =     615.79. Improved by 10.5102.
Iteration  6. Log joint probability =    622.993. Improved by 7.20259.
Iteration  7. Log joint probability =    629.992. Improved by 6.99954.
Iteration  8. Log joint probability =    631.131. Improved by 1.13813.
Iteration  9. Log joint probability =    631.866. I

...

Initial log joint probability = -6.39558
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       688.527     0.0455332       80.6023           1           1      111   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199        701.27     0.0167947       26.7395           1           1      229   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       703.231   0.000501713       17.0527      0.3932           1      353   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     375       704.594   0.000244913       19.9473   7.138e-06       0.001      509  LS failed, Hessian reset 
     399       704.871     0.0022011       19.9933       0.784       0.784      535   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     446       705.319   0.000130896    

INFO:prophet:Applying in parallel with <Client: 'tcp://129.82.208.160:8786' processes=24 threads=384, memory=1.47 TiB>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Making 3 forecasts with cutoffs between 2020-08-18 23:59:59.999999900 and 2021-03-06 23:59:59.999999900


Initial log joint probability = -6.39558
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       693.788     0.0160751        52.111           1           1      117   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       701.846   0.000600707       36.0867      0.2176      0.2176      237   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299         703.1    0.00122255       19.6191      0.4007      0.4007      360   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     397       705.171   0.000130808       12.3996   7.894e-06       0.001      549  LS failed, Hessian reset 
     399       705.206    0.00789727       25.9352           1           1      552   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       705.588   0.000762491    

INFO:prophet:Applying in parallel with <Client: 'tcp://129.82.208.160:8786' processes=24 threads=384, memory=1.47 TiB>


Time: 215.24625499296235
56, 

In [31]:
print(len(rmses))
print(len(changepoint_prior_scales))
print(len(seasonality_prior_scales))

56
56
56


In [21]:
pickle.dump(rmses, open('rmses.pkl', 'wb'))
pickle.dump(changepoint_prior_scales, open('changepoint_prior_scales.pkl', 'wb'))
pickle.dump(seasonality_prior_scales, open('seasonality_prior_scales.pkl', 'wb'))

In [26]:
df_parent_weights = pd.DataFrame(zip(parents, rmses, changepoint_prior_scales, seasonality_prior_scales),
                                columns=["GISJOIN", "rmse", "changepoint_prior_scale", "seasonality_prior_scale"])
df_parent_weights.to_csv('covid_parents_trained.csv', index=False)